In [2]:
import numpy as np
from scipy.stats import norm

def black_scholes_call(S, K, T, r, sigma):
    """
    Returns the Black–Scholes call price and Delta for a European call option.
    """
    # Avoid log(0) if S=0 or K=0 in edge cases
    S = np.array(S, dtype=float)
    K = np.array(K, dtype=float)
    T = np.array(T, dtype=float)
    r = np.array(r, dtype=float)
    sigma = np.array(sigma, dtype=float)

    sqrtT = np.sqrt(T)

    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * sqrtT)
    d2 = d1 - sigma * sqrtT

    # Call price
    call_price = S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)

    # Delta for a call is Phi(d1)
    delta = norm.cdf(d1)

    # Returns the bond position B that, combined with Delta shares of S, replicates the call option
    b_replicate = call_price - delta * S

    return call_price, delta, b_replicate


S = 100.0   # Underlying price
K = 100.0   # Strike price
T = 1.0     # 1 year to maturity
r = 0.05    # risk-free rate
sigma = 0.2 # volatility

call_price, delta, bond_position = black_scholes_call(S, K, T, r, sigma)
print("Call Price:", call_price)
print("Delta:", delta)

if bond_position >= 0:
    print(f"Bond position (B): {bond_position} (Invest in risk-free bond)")
else:
    print(f"Bond position (B): {bond_position} (Borrow at risk-free rate)")


Call Price: 10.450583572185565
Delta: 0.6368306511756191
Bond position (B): -53.232481545376345 (Borrow at risk-free rate)


In [24]:
import numpy as np
from scipy.stats import norm

def black_scholes_call(S: float, K: float, T: float, r: float, sigma: float) -> tuple[float, float]:
    """
    Returns the Black–Scholes call price and Delta for a European call option.

    Parameters
    ----------
    S: float Current stock price.
    K: float Strike price.
    T: float Time to maturity (in years).
    r: floatRisk-free interest rate (annualized).
    sigma: float Volatility of the underlying (annualized).

    Returns
    -------
    (call_price, delta): tuple of floats
        call_price : The Black–Scholes price of the call.
        delta      : The partial derivative of the call price wrt. the stock price.
    """
    # d1 and d2
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)

    # Call price
    call_price = S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)

    # Delta
    delta_call = norm.cdf(d1)

    return call_price, delta_call

def delta_hedged_short_call_example(S, K, T, r, sigma):
    """
    Demonstrates a delta-hedged short call using:
     - The short call option
     - A delta hedge in the underlying
     - A risk-free borrowing/lending position

    It sets up the position, so the initial net cash flow is zero,
    then calculates the final P&L at two potential stock prices at maturity
    and ignoring continuous balancing.
    """

    # 1) Compute call price and delta using Black–Scholes
    call_price, delta_call = black_scholes_call(S, K, T, r, sigma)

    print("=== Black–Scholes Computations ===")
    print(f"Underlying price (S0) = {S:.2f}")
    print(f"Strike (K)            = {K:.2f}")
    print(f"Time to maturity (T)  = {T:.2f} year(s)")
    print(f"Risk-free rate (r)    = {r:.2%}")
    print(f"Volatility (sigma)    = {sigma:.2%}\n")
    print(f"Call Price = ${call_price:.2f}")
    print(f"Call Delta = {delta_call:.3f}\n")

    # 2) Set up the delta-hedged short call

    # Short 1 call => collect premium
    premium_received = call_price

    # Buy delta_call shares => negative cashflow
    cost_shares = delta_call * S

    # Borrow the difference so initial net = 0
    borrowed = cost_shares - premium_received

    print("=== Position Setup (Initial) ===")
    print(f"Short 1 call: +${premium_received:.2f} premium")
    print(f"Buy {delta_call:.3f} shares: -${cost_shares:.2f}")
    print(f"Borrow cash: +${borrowed:.2f} (to net initial cashflow to $0.00)\n")

    # 3) Evaluate final payoff for two terminal prices
    #    ignoring continuous rebalancing

    terminal_prices = [90.0, 120.0]

    print("=== Payoff at Expiry (Discrete, No Rebalancing or Interest) ===")
    for S_T in terminal_prices:
        # Short call payoff: -(max(S_T - K, 0)) #plus the premium we initially collected
        call_exercise_value = max(S_T - K, 0.0)
        short_call_pnl = premium_received - call_exercise_value
        # short_call_pnl = call_exercise_value


        # Stock payoff: final stock value minus what we paid at inception
        stock_final_value = delta_call * S_T
        stock_pnl = stock_final_value - cost_shares

        # Bond/cash payoff: we owe exactly the borrowed amount
        # (ignoring interest)
        bond_pnl = borrowed - borrowed * (1+r)**T

        total_pnl = short_call_pnl + stock_pnl + bond_pnl

        print(f"Final Stock Price: S_T = ${S_T:.2f}")
        print(f"  - Short Call P&L  = ${short_call_pnl:.2f}")
        print(f"  - Stock P&L       = ${stock_pnl:.2f}")
        print(f"  - Bond (Borrowed) = ${bond_pnl:.2f}")
        print(f"  --> TOTAL P&L     = ${total_pnl:.2f}\n")


delta_hedged_short_call_example(
    S=100.0,
    K=100.0,
    T=1.0,
    r=0.05,
    sigma=0.25
)


=== Black–Scholes Computations ===
Underlying price (S0) = 100.00
Strike (K)            = 100.00
Time to maturity (T)  = 1.00 year(s)
Risk-free rate (r)    = 5.00%
Volatility (sigma)    = 25.00%

Call Price = $12.34
Call Delta = 0.627

=== Position Setup (Initial) ===
Short 1 call: +$12.34 premium
Buy 0.627 shares: -$62.74
Borrow cash: +$50.40 (to net initial cashflow to $0.00)

=== Payoff at Expiry (Discrete, No Rebalancing or Interest) ===
Final Stock Price: S_T = $90.00
  - Short Call P&L  = $12.34
  - Stock P&L       = $-6.27
  - Bond (Borrowed) = $-2.52
  --> TOTAL P&L     = $3.54

Final Stock Price: S_T = $120.00
  - Short Call P&L  = $-7.66
  - Stock P&L       = $12.55
  - Bond (Borrowed) = $-2.52
  --> TOTAL P&L     = $2.36



In [ ]:
## Black-Scholes American call option

**This code implements the Crank–Nicolson finite difference method to numerically solve the Black–Scholes PDE for valuing a European call option.**

It constructs a discretized grid for stock price and time, iterates backward in time to solve the option value at each point using a tridiagonal system (solved with the Thomas algorithm), and returns the option price for a given initial stock price \(S_0\). The main focus is on efficiently handling the boundary and terminal conditions for the PDE.

In [25]:
import numpy as np

def crank_nicolson_european_call(S0=100, K=100, T=1.0, r=0.05, sigma=0.2,
                                 Smax=300, M=200, N=200):
    """
    Solves the Black–Scholes PDE for a European call option
    using a Crank–Nicolson finite difference scheme.

    Parameters
    ----------
    S0    : float, initial stock price
    K     : float, strike price
    T     : float, time to maturity (years)
    r     : float, risk-free rate
    sigma : float, volatility
    Smax  : float, maximum grid value for S
    M     : int, number of spatial steps
    N     : int, number of time steps

    Returns
    -------
    price : float
        The computed option value at S=S0, t=0.
    """

    # 1) Set up grid in S (j=0..M) and time (n=0..N)
    #    We'll step in time backwards: from n=N => n=0
    dS = Smax / M
    dt = T / N

    # Stock price grid: S_j = j * dS
    S_values = np.linspace(0, Smax, M+1)

    # Option value grid: v[n, j] = V at time index n and stock index j
    # We store times going from 0..N, but we solve backwards from T to 0
    v = np.zeros((N+1, M+1))

    # 2) Terminal condition at t = T: v[N, j] = max(S_j - K, 0)
    for j in range(M+1):
        v[N, j] = max(S_values[j] - K, 0.0)

    # 3) Boundary conditions:
    #    a) S=0 => call value = 0
    #    b) S=Smax => call value ~ S - K e^{-r(T - t)}
    # We'll impose them at each time step
    # For Crank–Nicolson, we need them at steps n=1..N, j=0 or j=M

    # Coefficients for the PDE discretization:
    # alpha_j, beta_j, gamma_j for the second-order derivative terms, etc.
    # We'll create arrays to store them for j=1..M-1

    # r1 = r, since used in formula
    for n in reversed(range(N)):  # n = N-1 down to 0
        t_n = n * dt

        # Build the tri-diagonal system A * v_{n+1} = B * v_{n}
        # For Crank–Nicolson, we do 0.5 factor on each side
        # PDE: -r * v + ...

        # We will store sub-diagonal (a), diagonal (b), super-diagonal (c) for matrix
        a = np.zeros(M-1)
        b = np.zeros(M-1)
        c = np.zeros(M-1)
        rhs = np.zeros(M-1)

        for j in range(1, M):
            # j corresponds to S_j in [1..M-1]
            # Coeffs from PDE:
            sigma_sq = sigma*sigma
            A = 0.5 * sigma_sq * (j**2)  # because S_j = j*dS => S_j^2 => j^2 * dS^2
            B = 0.5 * r * j             # from r*S dV/dS => r * j
            # For standard PDE notation with dS, you might see:
            #   alpha = A - B
            #   beta  = -2*A - r1
            #   gamma = A + B
            # But we must incorporate dt/2 factors for CN.

            alpha = 0.5 * dt * (A - B)
            beta  = - dt * (A + r) + 2.0 * A * dt * 0.5  # We'll reorganize carefully
            gamma = 0.5 * dt * (A + B)

            # For Crank–Nicolson, the LHS matrix uses (1 + 0.5 * dt * PDE operator),
            # and the RHS uses (1 - 0.5 * dt * PDE operator).
            # Let's define:
            #  - on LHS: aL = -alpha, bL = 1 - beta, cL = -gamma
            #  - on RHS: aR = alpha,  bR = 1 + beta,  cR = gamma
            # This can be tricky to get correct sign and factor convention.
            # Let's follow a standard reference formula:

            aL = - alpha
            bL = 1 + dt*r + alpha + gamma
            cL = - gamma

            aR = alpha
            bR = 1 - dt*r - (alpha + gamma)
            cR = gamma

            # Build tri-diagonal for j in [1..M-1]
            if j == 1:
                a[j-1] = 0.0
            else:
                a[j-1] = aL

            b[j-1] = bL

            if j == M-1:
                c[j-1] = 0.0
            else:
                c[j-1] = cL

            # Right-hand side (RHS) contribution from v[n+1, j-1], v[n+1, j], v[n+1, j+1]
            # But we store them in "rhs" now for j in [1..M-1].
            # We'll set rhs[j-1] = bR * v[n+1, j] + aR * v[n+1, j-1] + cR * v[n+1, j+1].
            rhs_j = bR * v[n+1, j]
            if j > 1:
                rhs_j += aR * v[n+1, j-1]
            else:
                # j=1 => j-1=0 => boundary condition
                # v[n+1,0] = 0 => might skip
                pass
            if j < M-1:
                rhs_j += cR * v[n+1, j+1]
            else:
                # j=M-1 => j+1=M => boundary condition
                # v[n+1,M] ~ Smax - K e^{-r(T-(n+1)dt}
                bc_top = cR * (v[n+1, M])
                rhs_j += bc_top

            rhs[j-1] = rhs_j

        # Apply boundary conditions on v[n, 0] and v[n, M]
        # v[n,0] = 0
        # v[n,M] = S_values[M] - K * np.exp(-r*(T - t_n))

        # Solve tri-diagonal system A_mat * v_n_interior = rhs
        # We'll implement a simple Thomas algorithm:
        # We store solution in an array sol[0..M-1] => corresponds to j=1..M-1
        sol_interior = thomas_solve(a, b, c, rhs)

        # Put sol_interior back into v[n, j=1..M-1]
        for j in range(1, M):
            v[n, j] = sol_interior[j-1]

        # Enforce boundaries:
        v[n, 0] = 0.0  # for a call
        v[n, M] = S_values[M] - K * np.exp(-r*(T - t_n))

    # After finishing time steps, v[0, :] has the option values at t=0
    # We want the price at S0. So we find j* s.t. S_j is near S0
    # or do an interpolation:
    # For simplicity, pick nearest grid point:
    jstar = int(S0 / dS)
    price_approx = v[0, jstar]

    return price_approx

def thomas_solve(a, b, c, d):
    """
    Solve a tridiagonal system A x = d, where
    A = diag(b) + diag(a, -1) + diag(c, +1).
    a, b, c, d are 1D arrays of length n.
    This is the Thomas algorithm (TDMA).
    """
    n = len(d)
    # Forward eliminate
    for i in range(1, n):
        w = a[i] / b[i-1]
        b[i] -= w * c[i-1]
        d[i] -= w * d[i-1]
    # Back-substitution
    x = np.zeros(n)
    x[-1] = d[-1] / b[-1]
    for i in reversed(range(n-1)):
        x[i] = (d[i] - c[i]*x[i+1]) / b[i]
    return x

if __name__ == "__main__":
    price = crank_nicolson_european_call()
    print(f"European Call Price (Crank–Nicolson) = {price:.4f}")


European Call Price (Crank–Nicolson) = 9.3433


**This code calculates the price of an American call option using the Crank–Nicolson finite difference method to solve the Black–Scholes PDE. It iteratively builds a grid for option prices, enforces early exercise conditions, and computes the option value at a given initial stock price \(S_0\). The output is printed as "American Call Price (Crank–Nicolson)" with a numerical value.**

In [26]:
import numpy as np
def crank_nicolson_american_call(S0=100, K=100, T=1.0, r=0.05, sigma=0.2,
                                 Smax=300, M=200, N=200):
    """
    Solves the Black–Scholes PDE for an American call option
    using a Crank–Nicolson finite difference scheme.

    Parameters
    ----------
    S0    : float, initial stock price
    K     : float, strike price
    T     : float, time to maturity (years)
    r     : float, risk-free rate
    sigma : float, volatility
    Smax  : float, maximum grid value for S
    M     : int, number of spatial steps
    N     : int, number of time steps

    Returns
    -------
    price : float
        The computed option value at S=S0, t=0.
    """

    dS = Smax / M
    dt = T / N

    # Stock price grid: S_j = j * dS
    S_values = np.linspace(0, Smax, M + 1)
    # Option value grid: v[n, j] = V(t_n, S_j)
    v = np.zeros((N + 1, M + 1))

    # 1) Terminal condition at t = T
    for j in range(M + 1):
        v[N, j] = max(S_values[j] - K, 0.0)

    # 2) Crank–Nicolson loop (backwards in time)
    for n in reversed(range(N)):
        t_n = n * dt

        # Build system for interior points j = 1..M-1
        a = np.zeros(M - 1)  # sub-diagonal
        b = np.zeros(M - 1)  # diagonal
        c = np.zeros(M - 1)  # super-diagonal
        rhs = np.zeros(M - 1)

        for j in range(1, M):
            sigma_sq = sigma * sigma
            A = 0.5 * sigma_sq * (j ** 2)  # for second derivative in S
            B = 0.5 * r * j  # for first derivative in S

            alpha = 0.5 * dt * (A - B)
            beta = - dt * (A + r) + 2.0 * A * dt * 0.5
            gamma = 0.5 * dt * (A + B)

            aL = -alpha
            bL = 1 + dt * r + alpha + gamma
            cL = -gamma

            aR = alpha
            bR = 1 - dt * r - (alpha + gamma)
            cR = gamma

            if j == 1:
                a[j - 1] = 0.0
            else:
                a[j - 1] = aL

            b[j - 1] = bL

            if j == M - 1:
                c[j - 1] = 0.0
            else:
                c[j - 1] = cL

            rhs_j = bR * v[n + 1, j]
            if j > 1:
                rhs_j += aR * v[n + 1, j - 1]
            if j < M - 1:
                rhs_j += cR * v[n + 1, j + 1]
            else:
                # boundary at S=M
                rhs_j += cR * v[n + 1, M]

            rhs[j - 1] = rhs_j

        # Solve the tridiagonal system for the interior points
        sol_interior = thomas_solve(a, b, c, rhs)
        for j in range(1, M):
            v[n, j] = sol_interior[j - 1]

        # Boundary conditions
        v[n, 0] = 0.0
        v[n, M] = S_values[M] - K * np.exp(-r * (T - t_n))

        # 3) Early-exercise condition for American call:
        #    V(t_n, S_j) = max( V(t_n, S_j), S_j - K )
        for j in range(1, M):
            v[n, j] = max(v[n, j], S_values[j] - K)

    # Interpolate or pick nearest grid point for S0
    jstar = int(S0 / dS)
    price_approx = v[0, jstar]
    return price_approx


def thomas_solve(a, b, c, d):
    """
    Solve A x = d for a tridiagonal system:
    A = diag(b) + diag(a, -1) + diag(c, +1).
    """
    n = len(d)
    for i in range(1, n):
        w = a[i] / b[i - 1]
        b[i] -= w * c[i - 1]
        d[i] -= w * d[i - 1]
    x = np.zeros(n)
    x[-1] = d[-1] / b[-1]
    for i in reversed(range(n - 1)):
        x[i] = (d[i] - c[i] * x[i + 1]) / b[i]
    return x


if __name__ == "__main__":
    price = crank_nicolson_american_call()
    print(f"American Call Price (Crank–Nicolson) = {price:.4f}")

American Call Price (Crank–Nicolson) = 9.3433


# Willmot example EU Option

**This code defines a function `eu_option_value_3D` to calculate the value of a European option (Call/Put) using an explicit finite-difference method for solving the Black-Scholes PDE. It builds a price grid by iteratively working backward in time and applying the PDE's Theta calculation. The result is returned as a 2D NumPy array (`V`). Finally, the function is executed to compute and display a European call option price.**

In [ ]:
import numpy as np

def eu_option_value_3D(vol, int_rate, ptype, strike, expiration, NAS):
    """

    """

    # 1) Set up the asset grid and time step
    S  = np.zeros(NAS + 1)
    ds = 2.0 * strike / NAS   # "Infinity" set to twice the strike
    # A tentative dt based on stability:
    dt = 0.9 / (vol**2 * NAS**2)
    # Number of time steps, ensure integer so that T is exactly reached:
    NTS = int(expiration / dt) + 1
    dt = expiration / NTS     # adjusted dt

    # 2) Set up the value grid
    V = np.zeros((NAS + 1, NTS + 1))

    # Determine sign for call/put
    if ptype.upper() == 'P':
        q = -1.0  # put
    else:
        q =  1.0  # call

    # 3) Initialize asset values and terminal payoff at t = T (k = 0)
    for i in range(NAS + 1):
        S[i]   = i * ds
        payoff = q * (S[i] - strike)  # S-K for calls; K-S for puts
        V[i,0] = max(payoff, 0.0)

    # 4) Time-marching loop from k=1 to k=NTS
    #    We interpret k=0 as t=T (terminal condition),
    #    and k=NTS as t=0 (option value today).
    for k in range(1, NTS + 1):
        for i in range(1, NAS):  # interior points
            # Central differences
            dV_plus  = V[i+1, k-1]
            dV_minus = V[i-1, k-1]
            dV_center= V[i,   k-1]

            Delta = (dV_plus - dV_minus) / (2.0 * ds)
            Gamma = (dV_plus - 2.0 * dV_center + dV_minus) / (ds**2)

            # Theta from Black-Scholes PDE:
            #  Theta = -0.5 * sigma^2 * S^2 * Gamma - r * S * Delta + r * V
            #  and V(i, k) = V(i, k-1) - dt * Theta
            Theta = -0.5 * (vol**2) * (S[i]**2) * Gamma \
                    - int_rate * S[i] * Delta \
                    + int_rate * dV_center

            # Explicit step
            V[i, k] = dV_center - dt * Theta

        # 5) Boundary conditions at S=0 and S="infinity"
        #    (S=0) => V(0) decays like discounted payoff
        V[0,  k] = V[0,  k-1] * (1.0 - int_rate * dt)
        #    (S=max) => approximate by linear extrapolation
        V[NAS, k] = 2.0 * V[NAS-1, k] - V[NAS-2, k]

    return V
if __name__ == "__main__":
    price = eu_option_value_3D(
        vol=0.2,
        int_rate=0.05,
        ptype='C',
        strike=100,
        expiration=1.0,
        NAS=20
    )
    print(f"European Call Price (Explicit) = {price}")

**This notebook cell defines a function `eu_option_value_3D` to compute the value of a European option (Call or Put) using an explicit finite-difference method for solving the Black-Scholes equation. Parameters such as volatility, interest rate, strike price, and time to maturity are used to build a grid. The function iteratively calculates option prices at each time step, moving backward in time. Boundary conditions are applied at asset limits.**

**The executed code initializes parameters to compute an explicit grid-based European call price with specific values (e.g., volatility = 0.2, strike = 100), storing the output (`price`) in a 2D NumPy array.**

In [32]:
import numpy as np

def option_value_US(vol, int_rate, ptype, strike, expiration, etype, NAS):
    """
    Solve the Black–Scholes PDE on a grid for an American call or put
    using an explicit finite-difference scheme and an early-exercise check.

    Parameters
    ----------
    vol        : float
        Volatility (sigma)
    int_rate   : float
        Risk-free interest rate (r)
    ptype      : str
        'C' for call, 'P' for put
    strike     : float
        Strike price (K)
    expiration : float
        Time to maturity in years (T)
    etype      : str
        Exercise type, e.g. 'E' for European or 'A' for American.
        Only if etype == 'A' do we apply the early-exercise check.
    NAS        : int
        Number of steps in the asset (S) grid

    Returns
    -------
    V : 2D NumPy array of shape (NAS+1, NTS+1)
        Finite-difference solution for each asset node (rows) and
        time node (columns), going backward in time.
        V[i,0] = payoff at expiration.
        V[i, NTS] = value at time t=0.
    """

    # 1) Define the spatial (asset) step and initial time step
    S  = np.zeros(NAS + 1)
    ds = 2.0 * strike / NAS          # "Infinity" ~ 2*strike
    # Set a tentative dt for stability:
    dt = 0.9 / (vol**2 * NAS**2)
    NTS = int(expiration / dt) + 1   # number of time steps
    dt  = expiration / NTS           # adjust so that T is exactly reached

    # 2) Initialize arrays for the payoff and option value
    payoff = np.zeros(NAS + 1)
    V      = np.zeros((NAS + 1, NTS + 1))

    # Determine sign for call or put
    # q=+1 => call payoff = max(S - K, 0)
    # q=-1 => put payoff  = max(K - S, 0)
    q = 1.0 if ptype.upper() == 'C' else -1.0

    # 3) Set up the asset grid and the terminal payoff at t = T (k=0)
    for i in range(NAS + 1):
        S[i] = i * ds
        payoff_val = q * (S[i] - strike)  # S-K if call, K-S if put
        payoff[i]  = max(payoff_val, 0.0)
        # At maturity (k=0), the option value = payoff
        V[i, 0] = payoff[i]

    # 4) Time loop:  k=1..NTS  (k=0 is T;  k=NTS is time=0)
    for k in range(1, NTS + 1):
        # Interior asset points: i=1..(NAS-1)
        for i in range(1, NAS):
            dV_plus   = V[i+1, k-1]
            dV_center = V[i,   k-1]
            dV_minus  = V[i-1, k-1]

            # Finite differences
            Delta = (dV_plus - dV_minus) / (2.0 * ds)
            Gamma = (dV_plus - 2.0*dV_center + dV_minus) / (ds**2)

            # Black–Scholes "Theta" term:
            #   Theta = -0.5 * sigma^2 * S^2 * Gamma
            #           - r*S*Delta
            #           + r*V
            # Update with explicit Euler step: V(i,k) = V(i,k-1) - dt*Theta
            Theta = -0.5 * vol**2 * S[i]**2 * Gamma \
                    - int_rate * S[i] * Delta \
                    + int_rate * dV_center

            V[i, k] = dV_center - dt * Theta

        # 5) Boundary conditions
        # At S=0, approximate by discounting down one step:
        V[0,  k] = V[0,  k-1] * (1.0 - int_rate * dt)
        # At S = "infinity" ~ 2K, extrapolate linearly:
        V[NAS, k] = 2.0 * V[NAS-1, k] - V[NAS-2, k]

        # 6) Early exercise check (for American options)
        if etype.upper() == 'A':
            for i in range(NAS + 1):
                V[i, k] = max(V[i, k], payoff[i])

    # V[i, NTS] now holds the option value at time=0 for asset node i.
    return V
if __name__ == "__main__":
    price = option_value_US(
        vol=0.2,
        int_rate=0.05,
        ptype='C',
        strike=100,
        expiration=1.0,
        etype='A',
        NAS=20
    )
    print(f"European Call Price (Explicit) = {price}")

KeyboardInterrupt: 